In [1]:
import json

In [2]:
#CSV 파일 path 설정
import os
import csv
import pandas as pd
os.chdir(r'C:\Users\SSAFY\Desktop\crawling\hani')

In [3]:
# 전역변수 선언
article_data = []

# 뉴스 카테고리
news_category = ['politics','society', 'area', 'economy', 'international', 'culture', 'sports', 'science', 'animalpeople', 'opinion'] # 정치, 사회, 전국, 경제, 국제, 문화, 스포츠, 미래과학, 애니멀피플, 오피니언


In [4]:
#1 카테고리별 url 만들기
def make_hani_url(news_category_index, page):
    default_url = 'https://www.hani.co.kr/arti/'
    query_string = '?page='
    default_url += news_category[news_category_index]
    return default_url + query_string + str(page)

In [5]:
import requests
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

#2 카테고리 페이지 접근 후 링크 뽑아오기
# 최대 15개
def extract_hani_article_link(url):
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=header)
    html = response.text
    soup = bs(html, 'html.parser')
    a_elements = soup.select('li.ArticleList_item__jPMVW > article > a')
    links = []
    for element in a_elements:
        link = "https://www.hani.co.kr" + element.get('href')
        links.append(link)
    return links

In [6]:
import time
import random
# temp_url = 'https://www.hani.co.kr/arti/society/health/1130721.html'
#3. 해당 기사 링크 접속 후 아티클 정보 불러오기
def get_article(url):
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=header)
    html = response.text
    time.sleep(random.uniform(2, 4))
    soup = bs(html, 'html.parser')
    json_contents = soup.select('script#__NEXT_DATA__') # 아티클 Json에 해당하는 부분 가져오기
    json_content = json.loads(json_contents[0].get_text(strip=True)) #Json 형식으로 변경
    article = json_content.get('props', {}).get('pageProps', {}).get('article', {})
    return article


In [7]:
# 본문 HTML 태그 제거
import re
def delete_html_tag(context):
    processed_context = []
    for text in context:
        text = re.sub(r'<[%>]+Ws+(?=<)|<[^>]+>', '', text).strip()
        if text:
            processed_context.append(text)
        return  processed_context

In [8]:
#4. 불러온 기사 정보에서 필요한 정보를 뽑아 news_data 배열에 저장
def extract_article_info(url):
    article = get_article(url)
    
    # 제목
    title = article.get('title', {})

    # 카테고리 대분류
    section = article.get('section', {}).get('label', {})

    # 카테고리 중분류
    subSection = article.get('subSection', {}).get('label', {})

    # 작성일자
    createDate = article.get('createDate', {})

    # 본문
    content = article.get('content', {})

    # 사진
    imageList = article.get('imageList', {})

    # 첫번째 사진 url
    mainImgUrl = imageList[0].get('url') if imageList else None

    # 첫번째 사진 캡션
    mainImgCaption = imageList[0].get('caption') if imageList else None

    # 관련 기사 리스트
    relatedArticleList = article.get('relatedArticleList', {})
    
    news_info = {
        '대분류': section, 
        '중분류': subSection, 
        '제목': title, 
        '작성시간': createDate, 
        '썸네일 이미지': mainImgUrl, 
        '썸네일 캡션': mainImgCaption, 
        '본문': content, 
        '기사주소': url
    }
    time.sleep(random.uniform(1, 2))
    for i, relatedArticle in enumerate(relatedArticleList):
        relatedArticleTitle = '관련기사 제목' + str(i + 1)
        relatedArticleUrl = '관련기사 주소' + str(i + 1)
        relatedArticleImgUrl = '관련기사 이미지' + str(i + 1)
        news_info[relatedArticleTitle] = relatedArticle.get('title')
        news_info[relatedArticleUrl] = 'https://www.hani.co.kr' + relatedArticle.get('url')
        news_info[relatedArticleImgUrl] = relatedArticle.get('image', {}).get('url')
    return news_info
    

1. 필요한 라이브러리 임포트
-> 없으면 해당 라이브러리 pip install {추가해야하는 라이브러리 입력}
2. 두번째 블럭 csv 파일을 저장할 디렉토리 설정
-> os.chdir(r'C:\Users\SSAFY\Desktop\crawling\hani')
-> 해당 디렉토리에 csv 파일 저장
3. 맨 처음 블럭부터 ctrl + enter (notebook jupyter 사용하는 경우)
4. 두번째 블록의 카테고리 인덱스 확인하고 아래 #### 변수 값 변경

In [38]:

# 크롤링 시작점
# 1페이지에서 10페이지 전체 기사 출력하기
from datetime import datetime

##### 크롤링시 변경해야하는 값
start_page = 1 # 시작페이지
end_page = 10 # 크롤링 마지막 페이지
category_idx = 3 # 분야 카테고리  # 0:정치, 1:사회, 2:전국, 3:경제, 국제, 문화, 스포츠, 미래과학, 애니멀피플, 오피니언
#####

page = start_page
while page < end_page:
    try:
        article_data = []
        url = make_hani_url(category_idx, page)  # 경제 카테고리의 1페이지
        links = extract_hani_article_link(url)  # 1페이지 기사의 모든 url
        for link in links:
            news_info = extract_article_info(link)
            article_data.append(news_info)
        df = pd.DataFrame(article_data)
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name = f'article_{current_time}.csv'
        df.to_csv(file_name, index=False, encoding='utf-8')
        time.sleep(random.uniform(0, 1))
        page += 1
    except Exception as e:
        print(f"An error occurred: {e}")
        page -= 1